# Geospatial Data Analysis
## Excercise 5.2

In [8]:
import numpy as np
import os

### Input features import and reshaping

In [9]:
X = np.load(os.path.join("data", "partB", "indianpinearray.npy"))
print("Initial shape: ", X.shape)
X = X.reshape(-1,200)
print("Final shape: ", X.shape)

Initial shape:  (145, 145, 200)
Final shape:  (21025, 200)


### Ground truth import and reshaping

In [10]:
y = np.load(os.path.join("data", "partB", "IPgt.npy"))
print("Initial shape: ", y.shape)
y = y.reshape(-1)
print("Final shape: ", y.shape)
print("Classes: ", np.unique(y))

Initial shape:  (145, 145)
Final shape:  (21025,)
Classes:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]


There is also class 0 in the dataset. I remove class 0 which corresponds to unlabeled data.

In [11]:
X = X[y!=0]
y = y[y!=0]
print("Final shapes: ", X.shape, y.shape)
print("Classes: ", np.unique(y))

Final shapes:  (10249, 200) (10249,)
Classes:  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]


### Normalisation

I normalise X inside [0,1].

In [12]:
from sklearn import preprocessing
sc = preprocessing.MinMaxScaler()
X = sc.fit_transform(X)

### Imports

In [13]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

### Parameter tuning

At first we train the classifiers using the GridSearch algorithm so as to decide the best hyperparameter values based on accuracy. We try different hyperparameter values for both kernel SVM and RandomForest algorithms. I also split in training/ test set, equally balancing all classes and I use less samples for tuning given that especially the RF algorithm is really resource consuming. We also use stratified K-Fold in GridSearch so as to be sure that cross validation is performed with balanced splits using StratifiedKFold CV.

In [ ]:
# Train / Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=69, stratify=y)
#Set the parameters of each model by cross-validation gridsearch

names = ["SVM", "RandomForest"]
algorithms = [SVC(), RandomForestClassifier(n_jobs=-1)]
tuned_parameters = [{'kernel': ['rbf', 'linear'], 'gamma': ['scale', 'auto'], 'C': [10, 100, 1000]},
                    {'n_estimators': [200, 600], 'max_depth': [4, 10, None], 'min_samples_leaf': [1, 2, 5]}]

best_scores = []
params = []
kfolds = StratifiedKFold(3)
# Gridsearch loop for all classifiers
i=0
for (a, t_p) in list(zip(algorithms, tuned_parameters)):
    print("Tuning hyper-parameters, based on accuracy for: {}\nwith parameter choice:\n{}\n".format(names[i], t_p))
    clf = GridSearchCV(a, t_p, cv=kfolds.split(X_train, y_train), scoring = 'accuracy', n_jobs=-1)
    clf.fit(X_train, y_train)
    print("Mean performance of each parameter combination based on Cross Validation")
    performance = pd.DataFrame(clf.cv_results_['params'])
    performance["Score"] = clf.cv_results_['mean_test_score']
    print(performance)
    print("Best parameters set found on training set:")
    print(clf.best_params_)
    print("\nThe scores are computed on the full evaluation set for the best combination of parameters")
    #evaluate and store scores of estimators of each category on test set
    score = clf.score(X_test, y_test)
    print("Test score:", score)
    print("============================\n")
    #store round scores
    cv_scores.append(clf.cv_results_["mean_test_score"])
    params.append(clf.best_params_)
    best_scores.append(score)
    i+=1
final_scores = dict(zip(names, best_scores))
print(final_scores)

Tuning hyper-parameters, based on accuracy for: SVM
with parameter choice:
{'kernel': ['rbf', 'linear'], 'gamma': ['scale', 'auto'], 'C': [10, 100, 1000]}



### Pytorch

In [ ]:
# =============================================================================
# NumPy to Tensors
# =============================================================================
# Create tensors from numpy arrays
input = torch.Tensor(X_train).view(len(X_train),-1, 1)
target = torch.Tensor(y_train).view(len(X_train),-1, 1)
input_test = torch.Tensor(X_test).view(len(X_test),-1, 1)
target_test = torch.Tensor(y_test).view(len(y_test),-1, 1)

# =============================================================================
# Cuda
# =============================================================================
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
    input = input.cuda()
    target = target.cuda()
    input_test = input_test.cuda()
    target_test = target_test.cuda()
else:
    print("GPU not available, CPU used")
    device = torch.device("cpu")

In [ ]:
# =============================================================================
# Train and Evaluate the model
# =============================================================================
epochs = 200
epoch = 0
epoch_loss_val = []
epoch_loss_train = []
patience = 50 # initialise here
countdown = patience # Early stopping counter (epochs to wait)
while epoch < epochs and countdown > 0:
    # Training
    epoch +=1
    batch_loss = []
    # enumerate brings a batch of the data.
    for i, data in enumerate(trainloader):
        inputs = data['timeseries'].to(device) 
        labels, lengths = data['labels'].to(device), data['lengths'].to(device)
        optimizer.zero_grad()
        # forward + backward + optimize
        output = model(inputs, lengths)
        loss = criterion(output, labels)
        batch_loss.append(loss.item())
        loss.backward()
        optimizer.step()
    print("epoch: %d, loss: %1.5f" % (epoch, np.mean(batch_loss)))
    epoch_loss_train.append(np.mean(batch_loss))
    # Validation
    batch_loss = []
    batch_acc = []
    y_pred_val = []
    with torch.no_grad():
        for i, data in enumerate(valloader):
            inputs_val = data['timeseries'].to(device) 
            labels_val = data['labels'].to(device)
            lengths_val = data['lengths'].to(device)
            batch_pred = model(inputs_val, lengths_val)
            loss = criterion(batch_pred, labels_val)
            batch_loss.append(loss.item())
            batch_pred = [np.argmax(batch_pred[i].
                              to(torch.device('cpu')).
                              detach().numpy()) 
                          for i in range(len(batch_pred))]
            y_pred_val.append(batch_pred)
            batch_acc.append(metrics.
                             accuracy_score(labels_val.
                                            to(torch.device('cpu')).
                                            detach().numpy(), batch_pred))
    epoch_loss_val.append(np.mean(batch_loss))
    print("Validation loss: {:1.3f}, Validation Acc: {:1.3f}, Countdown: {} \n"
          .format(epoch_loss_val[-1], np.mean(batch_acc), countdown))
    # Early stopping condtion: N epochs without achieving loss than the
    # present minimum. No need to save models before patience
    if epoch_loss_val[-1] <= min(epoch_loss_val):
        countdown = patience #start countdown
    #checkpoint 
        if epoch >= patience: # no need to save before that
    #I ovewrite models so as to keep the last to trigger the countdown
            torch.save(model, os.path.join(os.getcwd(),
                      "models"+os.path.sep+best_model_name+".pt"))
    else:
        countdown -= 1
print("Finished Training!")

In [ ]:
# =============================================================================
# Plot Train / Validation Loss
# =============================================================================
plt.rcParams["figure.figsize"] = (20,6)
plt.figure()
plt.title("Relative Loss")
plt.plot(list(range(1,epoch+1)), epoch_loss_train, label='Training set')
plt.plot(list(range(1,epoch+1)), epoch_loss_val,  label='Validation set')
plt.grid()
plt.ylim(0, 0.5)
plt.legend(fancybox=True)
plt.show()

### Final evaluation

In [ ]:
###############################################################################
# Evaluation with various classification metrics (classification report)
###############################################################################
from confusion_matrix import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=69, stratify=y)

models = {'kNN': KNeighborsClassifier(n_neighbors=3),
          'NB': GaussianNB(),
          'MLP1':
          'MLP2':  }
# Create scores dictionary for each algorithm
scores=[]
mdl=[]
results=[]
for model in models.keys():
    clf = models[model]
    clf.fit(X_train,  y_train)
    mdl.append(model)
    y_pred = clf.predict(X_test)
    results.append((clf.score(X_test, y_test), y_pred))
    print (model, "\n")
    print(metrics.classification_report(y_test, y_pred, digits=5))
    cm = confusion_matrix(y_test, y_pred)
    plt.figure()
    plot_confusion_matrix(cm, classes=[0,1], title=model, cmap=plt.cm.Greens)
    print("True Positives: {}, False Positives: {}, True Negatives:, False Negatives: {} \n\n".format(cm[0,0], cm[0,1], cm[1,1], cm[1,0]))

In [ ]:
###############################################################################
# Evaluation with various classification metrics (classification report)
###############################################################################
from confusion_matrix import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

N=1000
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=69, stratify=y)
X_train, X_test, y_train, y_test = X_train[:N,:], X_test[:N,:], y_train[:N], y_test[:N]

models = {'kNN': KNeighborsClassifier(n_neighbors=3),
          'NB': GaussianNB(),
          'Perceptron': Perceptron(penalty='l1')}
# Create scores dictionary for each algorithm
scores=[]
mdl=[]
results=[]
for model in models.keys():
    clf = models[model]
    clf.fit(X_train,  y_train)
    mdl.append(model)
    y_pred = clf.predict(X_test)
    results.append((clf.score(X_test, y_test), y_pred))
    print (model, "\n")
    print(metrics.classification_report(y_test, y_pred, digits=5))
    cm = confusion_matrix(y_test, y_pred)
    plt.figure()
    plot_confusion_matrix(cm, classes=[0,1], title=model, cmap=plt.cm.Greens)
    print("True Positives: {}, False Positives: {}, True Negatives:, False Negatives: {} \n\n".format(cm[0,0], cm[0,1], cm[1,1], cm[1,0]))